In [1]:
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from evaluator.gpt_evaluator import FinancialDataProcessor
import time
import ast

/home/ubuntu/miniforge3/envs/multimodal/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
def wait_for_completion(job_id, processor, poll_interval=20):
    status = processor.check_status(job_id)
    while status.status not in ["completed", "failed"]:
        print(f"Current status: {status}. Waiting for {poll_interval} seconds...")
        time.sleep(poll_interval)
        status = processor.check_status(job_id)
    return status.status

# Process the financial Data

In [7]:
import os
import pandas as pd

def process_data(file_name, input_dir, output_dir):
    # Load the data
    df = pd.read_csv(os.path.join(input_dir, file_name))

    # Step 1: Filter out rows where 'text' starts with "Access to this page has been denied"
    df_filtered = df[~df['text'].str.startswith('Access to this page has been denied', na=False)].copy()

    # Step 2: Ensure the timestamp column is in datetime format
    df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])

    # Step 3: Convert the timestamp to just the date part
    df_filtered['date'] = df_filtered['timestamp'].dt.date

    # Step 4: Define URLs to match MarketWatch, Investors, and Reuters in order of priority
    url_priority = ['marketwatch', 'reuters', 'investors','Benzinga', 'zacks']
    
    # Step 5: Assign priority based on the URL. Check if the URL contains any of the specified keywords.
    df_filtered['priority'] = df_filtered['url'].apply(lambda x: next((i for i, kw in enumerate(url_priority) if kw in x), len(url_priority)))

    # Step 6: Sort by date and priority, so MarketWatch comes first, followed by Investors, then Reuters
    df_filtered = df_filtered.sort_values(by=['date', 'priority'])

    # Step 7: Group by date and sample one entry per day (from the highest-priority source)
    def sample_one_per_day(group):
        return group.groupby('priority', group_keys=False).apply(
            lambda x: x.sample(1)
        ).sort_values('priority').head(1)

    sampled_data = df_filtered.groupby('date', group_keys=False).apply(sample_one_per_day)

    # Step 8: Drop the 'priority' column used for sorting
    sampled_data = sampled_data.drop(columns=['priority'])

    # Step 9: Sort the sampled data by date
    sampled_data = sampled_data.sort_values(by='date')

    # Step 10: Save the resulting DataFrame to a new file
    output_file = os.path.join(output_dir, file_name)
    sampled_data.to_csv(output_file, index=False)


# Financial Data

In [5]:
def get_gpt_results(output_dir, results_dir, file_name):
    financial_data_processor = FinancialDataProcessor()

    data = pd.read_csv(os.path.join(output_dir, file_name))

    data = data.head(50)

    jsonl_path = os.path.join(results_dir, "batch.jsonl")
    output_path = os.path.join(results_dir, "data.txt")

    batch_object_id = financial_data_processor.create_and_run_batch_job(data, jsonl_path, ticker_column="ticker", date_column="timestamp", input_text_column="text")

    job_status = wait_for_completion(batch_object_id, financial_data_processor)

    print("Job status:", job_status)
    if job_status == "completed":
        print("Batch job completed successfully!")
        gpt_outputs = financial_data_processor.check_status_and_parse(batch_object_id , output_path)
        dict_list = [ast.literal_eval(item) for item in gpt_outputs]
        df = pd.DataFrame(dict_list)
        df['text'] = df.apply(lambda x: {"summary": x['summary'], "keywords": x['keywords']}, axis=1)
        df = df.drop(columns=['summary', 'keywords'])
        df.to_csv(os.path.join(results_dir, file_name), index=False)
    


In [4]:
financial_data_processor = FinancialDataProcessor()

financial_data_processor.gpt_call("hello", "AAPL", "2022-01-01")

'{"ticker":"AAPL","date":"2022-01-01","summary":"","keywords":""}'

In [8]:
input_dir = "/home/ubuntu/multimodal/Data/financial-raw/"
output_dir = "/home/ubuntu/multimodal/Data/financial-processed/"
results_dir = f"/home/ubuntu/multimodal/Data/financial-gpt/"

# file_names = ["AAPL.csv", "AMZN.csv", "AMD.csv", "AVGO.csv", "NFLX.csv"]
file_names = ["AVGO.csv"]
for file_name in file_names:
    process_data(file_name, input_dir, output_dir)
    get_gpt_results(output_dir, results_dir, file_name)

/tmp/ipykernel_1311234/918138568.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return group.groupby('priority', group_keys=False).apply(
/tmp/ipykernel_1311234/918138568.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return group.groupby('priority', group_keys=False).apply(
/tmp/ipykernel_1311234/918138568.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This 

batch job created with batch_object_id 
 batch_TD2b91r5DIdOxi4lRTgZkWrE
Current status: Batch(id='batch_TD2b91r5DIdOxi4lRTgZkWrE', completion_window='24h', created_at=1726067134, endpoint='/v1/chat/completions', input_file_id='file-55InbTHCJYBDjwipcLAneloh', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726153534, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Multimodal Forecasting'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)). Waiting for 20 seconds...
Job status: completed
Batch job completed successfully!


In [6]:
import pandas as pd
import glob

# Step 1: Retrieve all CSV file paths in the specified directory
file_paths = glob.glob('/home/ubuntu/multimodal/Data/financial-gpt/*.csv')

# Step 2: Read all CSV files and concatenate them into a single DataFrame
all_data = pd.concat((pd.read_csv(file) for file in file_paths))

# Step 3: Ensure the 'date' column is in datetime format for proper grouping
all_data['date'] = pd.to_datetime(all_data['date'])

# Step 4: Replace NaN in the 'text' column with blank strings
all_data['text'] = all_data['text'].fillna('')

# Step 5: Group by the 'date' column and concatenate the 'text' column for each group
grouped_data = all_data.groupby('date').agg({'text': '\n'.join})

# Step 6: Reset the index to flatten the DataFrame
grouped_data = grouped_data.reset_index()

# Step 7: Save the aggregated data to a CSV file
grouped_data.to_csv('results_grouped_by_date.csv', index=False)

# Print the shape of the resulting DataFrame
print(grouped_data.shape)
print("Data grouped by date and saved to 'results_grouped_by_date.csv'")


(791, 2)
Data grouped by date and saved to 'results_grouped_by_date.csv'
